In [1]:
#import dependencies

import tweepy
import config
import numpy as np
import re
import pandas as pd

In [2]:
twitter_auth = tweepy.OAuthHandler(config.consumer_key, config.consumer_key_secret)
api = tweepy.API(twitter_auth)

tweets = api.search("data", lang='en', count=200) # Search term goes here

print(tweets)

[Status(_api=<tweepy.api.API object at 0x0000021E4FC15C88>, _json={'created_at': 'Tue Jul 14 23:11:43 +0000 2020', 'id': 1283177386335506433, 'id_str': '1283177386335506433', 'text': 'RT @RabbiJill: My husband, with a masters in public health, said upon hearing this- this is so trump admin can manipulate the data #CDC htt…', 'truncated': False, 'entities': {'hashtags': [{'text': 'CDC', 'indices': [131, 135]}], 'symbols': [], 'user_mentions': [{'screen_name': 'RabbiJill', 'name': 'Rabbi Jill Zimmerman', 'id': 26306441, 'id_str': '26306441', 'indices': [3, 13]}], 'urls': []}, 'metadata': {'iso_language_code': 'en', 'result_type': 'recent'}, 'source': '<a href="http://twitter.com/#!/download/ipad" rel="nofollow">Twitter for iPad</a>', 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 593959216, 'id_str': '593959216', 'name': 'Jett  🌊', 'screen_name': 'jettward', 'l

In [3]:



df = pd.DataFrame(columns=('tweet_text', 'user_location', 
                           'user_followers_count', 'user_friends_count',
                           'user_account_age', 'user_verified',
                           'user_favourites_count', 'user_tweets',
                           'tweet_retweeted', 'tweet_retweet_count', 'tweet_favorite_count'))

# Remove duplicates
df.sort_values("tweet_text", inplace = True) 
df.drop_duplicates(subset ="tweet_text", keep = False, inplace = True) 

for tweet in tweets:

    df = df.append({'user_id': tweet.user.id_str,
                    'user': tweet.user.screen_name,
                    'tweet_text': re.sub(r'http\S+', '', tweet.text), #Remove links
                    'tweet_date': tweet.created_at,
                    'user_location': tweet.user.location,
                    'user_followers_count': tweet.user.followers_count, 
                    'user_friends_count': tweet.user.friends_count,
                    'user_account_age': tweet.user.created_at, 
                    'user_verified': tweet.user.verified,
                    'user_favourites_count': tweet.user.favourites_count,
                    'user_tweets': tweet.user.statuses_count,
                    'tweet_retweeted': tweet.retweeted,
                    'tweet_retweet_count': tweet.retweet_count,
                    'tweet_favorite_count': tweet.favorite_count},
                   ignore_index=True)

df.head()

,tweet_text,user_location,user_followers_count,user_friends_count,user_account_age,user_verified,user_favourites_count,user_tweets,tweet_retweeted,tweet_retweet_count,tweet_favorite_count,tweet_date,user,user_id
0,"RT @RabbiJill: My husband, with a masters in p...","Troy, OH",3267,3102,2012-05-29 19:04:54,False,14237,55252,False,309,0,2020-07-14 23:11:43,jettward,593959216
1,RT @JackDMurphy: Remember Tom? Remember how he...,,311,184,2012-05-29 15:45:42,False,2614,72348,False,58991,0,2020-07-14 23:11:43,derp_gir,593813725
2,RT @mubeatTV: [#MubeatVote 📈] (20.07.02 ~ 20.0...,190629 neo city,254,256,2016-07-01 19:05:30,False,152342,35467,False,435,0,2020-07-14 23:11:42,cixsation,748955706150256640
3,RT @netblocks: Confirmed: Significant disrupti...,"Melbourne, Victoria",220,105,2019-09-29 14:53:26,False,4258,2263,False,1305,0,2020-07-14 23:11:42,ghashangkhanoom,1178321835898703875
4,@Lukewearechange I use a netgear unite explore...,,32,181,2015-03-28 04:07:51,False,386,613,False,0,0,2020-07-14 23:11:42,mawic5150,3122961611


In [4]:
df=df[['user_id', 'tweet_date', 'user', 'tweet_text']]
df.head()

,user_id,tweet_date,user,tweet_text
0,593959216,2020-07-14 23:11:43,jettward,"RT @RabbiJill: My husband, with a masters in p..."
1,593813725,2020-07-14 23:11:43,derp_gir,RT @JackDMurphy: Remember Tom? Remember how he...
2,748955706150256640,2020-07-14 23:11:42,cixsation,RT @mubeatTV: [#MubeatVote 📈] (20.07.02 ~ 20.0...
3,1178321835898703875,2020-07-14 23:11:42,ghashangkhanoom,RT @netblocks: Confirmed: Significant disrupti...
4,3122961611,2020-07-14 23:11:42,mawic5150,@Lukewearechange I use a netgear unite explore...


In [5]:
#Remove handles

def remove_pattern(input_txt, pattern):
  r = re.findall(pattern, input_txt)
  for i in r:
    input_txt = re.sub(i, '', input_txt)

  return input_txt

df['tidy_tweet'] = np.vectorize(remove_pattern)(df['tweet_text'], "@[\w]*")

df.head()

,user_id,tweet_date,user,tweet_text,tidy_tweet
0,593959216,2020-07-14 23:11:43,jettward,"RT @RabbiJill: My husband, with a masters in p...","RT : My husband, with a masters in public heal..."
1,593813725,2020-07-14 23:11:43,derp_gir,RT @JackDMurphy: Remember Tom? Remember how he...,RT : Remember Tom? Remember how he just sold h...
2,748955706150256640,2020-07-14 23:11:42,cixsation,RT @mubeatTV: [#MubeatVote 📈] (20.07.02 ~ 20.0...,RT : [#MubeatVote 📈] (20.07.02 ~ 20.07.08 data...
3,1178321835898703875,2020-07-14 23:11:42,ghashangkhanoom,RT @netblocks: Confirmed: Significant disrupti...,RT : Confirmed: Significant disruption to mult...
4,3122961611,2020-07-14 23:11:42,mawic5150,@Lukewearechange I use a netgear unite explore...,I use a netgear unite explore on art network ...


In [6]:
#Remove special characters, number, and punctuation

df['tidy_tweet'] = df['tidy_tweet'].str.replace("[^a-zA-Z#]", " " )
df.head()

,user_id,tweet_date,user,tweet_text,tidy_tweet
0,593959216,2020-07-14 23:11:43,jettward,"RT @RabbiJill: My husband, with a masters in p...",RT My husband with a masters in public heal...
1,593813725,2020-07-14 23:11:43,derp_gir,RT @JackDMurphy: Remember Tom? Remember how he...,RT Remember Tom Remember how he just sold h...
2,748955706150256640,2020-07-14 23:11:42,cixsation,RT @mubeatTV: [#MubeatVote 📈] (20.07.02 ~ 20.0...,RT #MubeatVote data...
3,1178321835898703875,2020-07-14 23:11:42,ghashangkhanoom,RT @netblocks: Confirmed: Significant disrupti...,RT Confirmed Significant disruption to mult...
4,3122961611,2020-07-14 23:11:42,mawic5150,@Lukewearechange I use a netgear unite explore...,I use a netgear unite explore on art network ...


In [7]:
#Remove short words

df['tidy_tweet'] = df['tidy_tweet'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>2]))
df.head()

,user_id,tweet_date,user,tweet_text,tidy_tweet
0,593959216,2020-07-14 23:11:43,jettward,"RT @RabbiJill: My husband, with a masters in p...",husband with masters public health said upon h...
1,593813725,2020-07-14 23:11:43,derp_gir,RT @JackDMurphy: Remember Tom? Remember how he...,Remember Tom Remember how just sold his share ...
2,748955706150256640,2020-07-14 23:11:42,cixsation,RT @mubeatTV: [#MubeatVote 📈] (20.07.02 ~ 20.0...,#MubeatVote data The Legendary Cover Stage bas...
3,1178321835898703875,2020-07-14 23:11:42,ghashangkhanoom,RT @netblocks: Confirmed: Significant disrupti...,Confirmed Significant disruption multiple netw...
4,3122961611,2020-07-14 23:11:42,mawic5150,@Lukewearechange I use a netgear unite explore...,use netgear unite explore art network with hac...


In [8]:
#write results to a csv
df.to_csv("tweets_v2.csv", encoding='utf-8-sig', index=False)